In [1]:
import gc
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from glob import glob
from tqdm import tqdm
from keras import backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras.preprocessing import image                  
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

In [2]:
def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [4]:
diseases = [
    'Cardiomegaly','Emphysema','Effusion',
    'Hernia','Nodule','Pneumothorax',
    'Atelectasis','Pleural_Thickening',
    'Mass','Edema','Consolidation',
    'Infiltration','Fibrosis','Pneumonia'
]

# dataset_df = pd.read_csv('/kaggle/input/data/Data_Entry_2017.csv')
dataset_df = pd.read_csv('./dataset_information/Data_Entry_2017.csv')

In [5]:
# Applying One Hot Encoding to Labels
for disease in diseases:
    dataset_df[disease] = dataset_df['Finding Labels'].apply(lambda x: 1 if disease in x else 0)

In [6]:
image_labels = dataset_df[diseases].to_numpy()
image_paths = {
    # os.path.basename(x): x for x in glob(os.path.join('..', 'input', 'data', 'images*', 'images', '*.png'))
    os.path.basename(x): x for x in glob(os.path.join('.', 'images', '*.png'))
}

print(f"Samples Found: {len(image_paths)}")

Samples Found: 112120


In [7]:
# Storing path to each image against image name in the dataframe
dataset_df['Image Path'] = dataset_df['Image Index'].map(image_paths.get)

In [8]:
images_list = dataset_df['Image Path'].tolist()

labelB = (dataset_df[diseases].sum(axis = 1) > 0).tolist()
labelB = np.array(labelB, dtype = int)

In [9]:
def read_image_to_tensor(path, shape):
    # Loads RGB image to PIL format
    img = image.load_img(path, target_size = shape)
    
    # Convert PIL image to 3D tensor of specific shape
    # and normalizes it by dividing each pixel by 255
    normalized_image_tensor = image.img_to_array(img) / 255
    
    # Convert 3D tensor to 4D tensor with specific shape 
    # (1, shape, 3) and return it
    return np.expand_dims(normalized_image_tensor, axis = 0)

In [10]:
def image_to_array(paths, shape):
    images_arrays = [read_image_to_tensor(path, shape) for path in tqdm(paths, desc = "Progress", ncols = 100)]
    return np.vstack(images_arrays)

In [11]:
# Defining some hyper-parameters
IMAGE_SHAPE = (75, 75)
EPOCHS = 5
BATCH_SIZE = 32

In [12]:
# Samples in Training Set: 70%
# Samples in Validation Set: 10%

# Storing labels of samples for each set
train_labels = labelB[ : 78484][ : , np.newaxis]
valid_labels = labelB[78484 : 89696][ : , np.newaxis]

# Storing arrays of samples for each set
training_samples = image_to_array(images_list[ : 78484], shape = IMAGE_SHAPE)
validation_samples = image_to_array(images_list[78484 : 89696], shape = IMAGE_SHAPE)

Progress: 100%|███████████████████████████████████████████████| 85000/85000 [23:00<00:00, 61.57it/s]

Progress: 100%|███████████████████████████████████████████████| 10000/10000 [02:37<00:00, 63.63it/s]

Progress: 100%|███████████████████████████████████████████████| 17120/17120 [04:35<00:00, 62.13it/s]


In [13]:
# Creating a model with MobileNetV2 as base.
m_net = MobileNetV2(
    weights = 'imagenet',
    include_top = False,
    input_shape = training_samples.shape[1 : ]
)

custom_classifier = Sequential()
custom_classifier.add(GlobalAveragePooling2D(input_shape = m_net.output_shape[1 : ]))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(50, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(1, activation = 'sigmoid'))

model = Model(inputs = m_net.input, outputs = custom_classifier(m_net.output))
# model.summary()


258080768/258076736 [==============================] - 538s 2us/step

258088960/258076736 [==============================] - 538s 2us/step


In [14]:
# Defining 2 optimizers to test the model with.

SGD_optimizer = tf.keras.optimizers.SGD(
    learning_rate = 1e-4, 
    decay = 1e-6, 
    momentum = 0.9, 
    nesterov = True
)
adam_optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
)

In [15]:
# Defining object for augmentation 

train_datagen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False 
)

In [ ]:
log = CSVLogger('saved_models/log_pretrained_CNN.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/pretrainedVGG.best.from_scratch.hdf5', verbose=1, save_best_only=True)

In [17]:
# Compiling model with loss function, optimizer and metrics

model.compile(
    optimizer = SGD_optimizer,
    loss = 'binary_crossentropy',
    metrics = [
        'accuracy',
        precision_threshold(threshold = 0.5), 
        recall_threshold(threshold = 0.5), 
        fbeta_score_threshold(beta=0.5, threshold = 0.5)
    ]
)

In [19]:
%%timeit -n1 -r1

history = model.fit_generator(
    train_datagen.flow(
        training_samples, 
        train_labels, 
        batch_size = BATCH_SIZE
    ),
    steps_per_epoch = len(training_samples) // BATCH_SIZE,
    validation_data = (validation_samples, valid_labels),
    validation_steps = len(validation_samples) // BATCH_SIZE,
    epochs = EPOCHS,
    callbacks = [checkpointer], 
    verbose = 1
)

Epoch 1/5

2656/2656 [==============================] - 1466s 552ms/step - loss: 0.6882 - accuracy: 0.5413 - precision: 0.4900 - recall: 0.1516 - fbeta_score: 0.3195

Epoch 2/5

2656/2656 [==============================] - 1443s 543ms/step - loss: 0.6868 - accuracy: 0.5451 - precision: 0.5035 - recall: 0.1632 - fbeta_score: 0.3356

Epoch 3/5

2656/2656 [==============================] - 1525s 574ms/step - loss: 0.6837 - accuracy: 0.5556 - precision: 0.5391 - recall: 0.1837 - fbeta_score: 0.3687

Epoch 4/5

2656/2656 [==============================] - 1500s 565ms/step - loss: 0.6793 - accuracy: 0.5654 - precision: 0.5579 - recall: 0.2279 - fbeta_score: 0.4159

Epoch 5/5

2656/2656 [==============================] - 1464s 551ms/step - loss: 0.6749 - accuracy: 0.5815 - precision: 0.5755 - recall: 0.3232 - fbeta_score: 0.4857

2h 3min 19s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
# Freeing Memory with Python's Garbase Collector
# del training_samples
# del validation_samples
# del train_labels
# del valid_labels
# gc.collect()

In [ ]:
# Samples in Testing Set: 20%
test_labels = labelB[89696 : ][ : , np.newaxis]
test_samples = image_to_array(images_list[89696 : ], shape = IMAGE_SHAPE)

prediction = model.predict(test_samples)

In [ ]:
# Evaluation of Trained Model
threshold = 0.5
beta = 0.5

accuracy = K.eval(binary_accuracy(K.variable(value=test_labels), K.variable(value=prediction)))
precision = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
recall = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
f1_score = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))

print (f"Accuracy: {accuracy} \nRecall: {recall} \nPrecision: {precision} \nF1-Score: {f1_score}")